# Slim

- Keras 와 비슷한 점을 지향
- network 구조를 효율적으로 사용하기 위해 만들어짐

### 그럼 왜 keras를 안쓸고 slim을 사용할까?

- slim의 독특한 특징
- slim base의 pre-trained model 이 많음

### 실습 시작

In [1]:
import tensorflow as tf
# contrib는  Tensorflow 1v 베타  2v부터는 사라짐
slim = tf.contrib.slim

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



- device 선택하기

In [2]:
slim.variable

<function tensorflow.contrib.framework.python.ops.variables.variable(name, shape=None, dtype=None, initializer=None, regularizer=None, trainable=True, collections=None, caching_device=None, device=None, partitioner=None, custom_getter=None, use_resource=None, synchronization=<VariableSynchronization.AUTO: 0>, aggregation=<VariableAggregation.NONE: 0>)>

In [45]:
tf.reset_default_graph()
# tf
with tf.device("/cpu:0"):
    weights = tf.Variable(tf.truncated_normal(shape=[10,10,3,3]),name="weights")

# slim
weights = slim.variable("weight", shape=[10,10,3,3], initializer=tf.truncated_normal_initializer,device="/cpu:0")

### Variable 의 종류
1. variable
모델을 저장하고 싶을때, save를 통해서 저장이 되는 변수
2. local variable
session이 살아있는 동안만 존재하는 변수

3. slim 은 model variable 을 하나더 만듦

In [14]:
tf.reset_default_graph()
weights = slim.model_variable("weights", shape=[10,10,3,3], initializer=tf.truncated_normal_initializer,device="/cpu:0")
weights = slim.variable("my_var", shape=[10,10,3,3], initializer=tf.truncated_normal_initializer,device="/cpu:0")
print(slim.get_model_variables()) # 모델 변수 호출
print(slim.get_variables()) #모든 변수 호출

[<tf.Variable 'weights:0' shape=(10, 10, 3, 3) dtype=float32_ref>]
[<tf.Variable 'weights:0' shape=(10, 10, 3, 3) dtype=float32_ref>, <tf.Variable 'my_var:0' shape=(10, 10, 3, 3) dtype=float32_ref>]


### slim 의 layer

In [21]:
# 에러가 나는 경우 pip install gast==0.2.2
tf.reset_default_graph()
net = tf.placeholder(tf.float32,[16,32,32,256])
with tf.variable_scope("repeat_1"):
    net = slim.conv2d(net,256,[3,3],scope="conv3_1")
    net = slim.conv2d(net,256,[3,3],scope="conv3_2")
    net = slim.conv2d(net,256,[3,3],scope="conv3_3")
    net = slim.max_pool2d(net,[2,2],scope="pool2")
# 겹치는 모델 설정을 scope 범위를 사용하여 지정할 수 있음
tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope="repeat_1")

[<tf.Variable 'repeat_1/conv3_1/weights:0' shape=(3, 3, 256, 256) dtype=float32_ref>,
 <tf.Variable 'repeat_1/conv3_1/biases:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'repeat_1/conv3_2/weights:0' shape=(3, 3, 256, 256) dtype=float32_ref>,
 <tf.Variable 'repeat_1/conv3_2/biases:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'repeat_1/conv3_3/weights:0' shape=(3, 3, 256, 256) dtype=float32_ref>,
 <tf.Variable 'repeat_1/conv3_3/biases:0' shape=(256,) dtype=float32_ref>]

In [23]:
tf.reset_default_graph()
net = tf.placeholder(tf.float32,[16,32,32,256])
with tf.variable_scope("repeat_1"):
    # + 반복 모델을 for문대신 사용하여 편하게 사용 가능
    net = slim.repeat(net,3,slim.conv2d,256,[3,3],scope="conv3")
    net = slim.max_pool2d(net,[2,2],scope="pool2")
# 겹치는 모델 설정을 scope 범위를 사용하여 지정할 수 있음
tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope="repeat_1")

[<tf.Variable 'repeat_1/conv3/conv3_1/weights:0' shape=(3, 3, 256, 256) dtype=float32_ref>,
 <tf.Variable 'repeat_1/conv3/conv3_1/biases:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'repeat_1/conv3/conv3_2/weights:0' shape=(3, 3, 256, 256) dtype=float32_ref>,
 <tf.Variable 'repeat_1/conv3/conv3_2/biases:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'repeat_1/conv3/conv3_3/weights:0' shape=(3, 3, 256, 256) dtype=float32_ref>,
 <tf.Variable 'repeat_1/conv3/conv3_3/biases:0' shape=(256,) dtype=float32_ref>]

- 파라미터가 다른경우의 repet

In [27]:
tf.reset_default_graph()
net = tf.placeholder(tf.float32,[16,32,32,256])
with tf.variable_scope("repeat_1"):
    # + 반복 모델을 for문대신 사용하여 편하게 사용 가능
    net = slim.stack(net,slim.conv2d,[(32,[3,3]),(32,[2,2]),(64,[3,3]),(64,[2,2])],scope="conv3")
    net = slim.max_pool2d(net,[2,2],scope="pool2")
# 겹치는 모델 설정을 scope 범위를 사용하여 지정할 수 있음
tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope="repeat_1")

[<tf.Variable 'repeat_1/conv3/conv3_1/weights:0' shape=(3, 3, 256, 32) dtype=float32_ref>,
 <tf.Variable 'repeat_1/conv3/conv3_1/biases:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'repeat_1/conv3/conv3_2/weights:0' shape=(2, 2, 32, 32) dtype=float32_ref>,
 <tf.Variable 'repeat_1/conv3/conv3_2/biases:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'repeat_1/conv3/conv3_3/weights:0' shape=(3, 3, 32, 64) dtype=float32_ref>,
 <tf.Variable 'repeat_1/conv3/conv3_3/biases:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'repeat_1/conv3/conv3_4/weights:0' shape=(2, 2, 64, 64) dtype=float32_ref>,
 <tf.Variable 'repeat_1/conv3/conv3_4/biases:0' shape=(64,) dtype=float32_ref>]

- scope
layer를 쌓으면서 겹치는 argument가 많은경우

In [37]:
tf.reset_default_graph()
padding = "SAME"
initializer = tf.truncated_normal_initializer(stddev=0.01)
regularizer = slim.l2_regularizer(0.0005)

inputs = tf.placeholder(tf.float32,[16,224,224,3])
ner = slim.conv2d(inputs,64,[11,11],4,padding=padding, weights_regularizer=regularizer, weights_initializer=initializer,scope="conv1")
ner = slim.conv2d(ner,64,[11,11],128,padding=padding, weights_regularizer=regularizer, weights_initializer=initializer,scope="conv2")
ner = slim.conv2d(ner,64,[11,11],256,padding=padding, weights_regularizer=regularizer, weights_initializer=initializer,scope="conv3")

In [44]:
# slim으로 교체
tf.reset_default_graph()
inputs = tf.placeholder(tf.float32,[16,224,224,3])
with slim.arg_scope([slim.conv2d],padding="SAME",
                  weights_regularizer=slim.l2_regularizer(0.0005),
                  weights_initializer=tf.truncated_normal_initializer(stddev=0.01)):
    ner = slim.conv2d(inputs,64,[11,11],4,scope="conv1")
    ner = slim.conv2d(ner,64,[11,11],128,scope="conv2")
    ner = slim.conv2d(ner,64,[11,11],256,scope="conv3")

scope 는 2중 3중으로도 가능하다 <br>
코드 설명 생략

### slim의 training

- loss의 더하기  전부 가져오기 가능
ex) <br>
slim.losses.get_total_loss(add_regularization_losses=False) <br>

- train op 과 update op
트레이닝(백프로파게이션)으로 업데이트 하지 않는 파라미터의 경우, tf에서는 control flow를 이용하여 control dependency를 통하여 업데이트를 할 수 있다. <br>
-> 난이도가 매우 어려움  <br>
slim.learning.create_train_op(total_loss, optimizer) -> 두가지를 동시에 사용 <br>

=> Tensorflow v2 에서는 해당 기능이 필요하지 않음 <br>

- Fine-Tuning <br>
내가 트레이닝한 그래프와 다른사람이 트레이닝한 그래프가 일치하지 않는경우 ... <br>
variable restore할지를 결정해야 하기 때문에 variable name을 하나씩 가져와야함  <br>
slim 은 여러가지 가져올 수 있는 방법을 제공 <br>
slim.get_variables_by_name <br> 
slim.get_variables_by_suffix <br>
get_variables <br>
get_variables_to_restore(include=[" 값 "])  <br>
get_variables_to_restore(exclude=[" 값 "]) <br>